In [1]:
import io
import pathlib
import urllib.request
import zipfile

import numpy as np
import pandas as pd

# Background

Summary file 1:
https://www.census.gov/prod/cen2010/doc/sf1.pdf

Data File Segement contains population table 3 "P3" to population table 9 "P9". "P" prefixes denote census block population tables. "PCT" prefixes denote census tract tables. The target tagble is P5 (census doc page 184)

Census file 3:
    P3. RACE Universe: Total population (8)
    P4. HISPANIC OR LATINO ORIGIN Universe: Total population (3)
    P5. HISPANIC OR LATINO ORIGIN BY RACE Universe: Total population (17)

P5 data:
    Total: P0050001 03 9 
        Not Hispanic or Latino: P0050002 03 9 
            White alone P0050003 03 9 
            Black or African American alone P0050004 03 9 
            American Indian and Alaska Native alone P0050005 03 9 
            Asian alone P0050006 03 9 
            Native Hawaiian and Other Pacific Islander alone P0050007 03 9 
            Some Other Race alone P0050008 03 9 
            Two or More Races P0050009 03 9 
        Hispanic or Latino: P0050010 03 9 
            White alone P0050011 03 9 
            Black or African American alone P0050012 03 9 
            American Indian and Alaska Native alone P0050013 03 9 
            Asian alone P0050014 03 9 
            Native Hawaiian and Other Pacific Islander alone P0050015 03 9 
            Some Other Race alone P0050016 03 9 
            Two or More Races P0050017 03 9

Levels:
871 State-5-Digit ZIP Code Tabulation Area 
881 State-5-Digit ZIP Code Tabulation Area-County

All hispanic aggregated
Some other race alone (iterative prop)
Asian and API combined

In [4]:
# https://www.census.gov/prod/cen2010/doc/sf2.pdf
GEO_MAP_2010 = {
    'FILEID'  : (1  , 7  ),
    'STUSAB'  : (7  , 9  ),
    'SUMLEV'  : (9  , 12 ),
    'GEOCOMP' : (12 , 14 ),
    'CHARITER': (14 , 17 ),
    'CIFSN'   : (17 , 19 ),
    'LOGRECNO': (19 , 26 ),
    'REGION'  : (26 , 27 ),
    'DIVISION': (27 , 28 ),
    'STATE'   : (28 , 30 ),
    'COUNTY'  : (30 , 33 ),
    'COUNTYCC': (33 , 35 ),
    'COUNTYSC': (35 , 37 ),
    'COUSUB'  : (37 , 42 ),
    'COUSUBCC': (42 , 44 ),
    'COUSUBSC': (44 , 46 ),
    'PLACE'   : (46 , 51 ),
    'PLACECC' : (51 , 53 ),
    'PLACESC' : (53 , 55 ),
    'TRACT'   : (55 , 61 ),
    'BLKGRP'  : (61 , 62 ),
    'BLOCK'   : (62 , 66 ),
    'IUC'     : (66 , 68 ),
    'CONCIT'  : (68 , 73 ),
    'CONCITCC': (73 , 75 ),
    'CONCITSC': (75 , 77 ),
    'AIANHH'  : (77 , 81 ),
    'AIANHHFP': (81 , 86 ),
    'AIANHHCC': (86 , 88 ),
    'AIHHTLI' : (88 , 89 ),
    'AITSCE'  : (89 , 92 ),
    'AITS'    : (92 , 97 ),
    'AITSCC'  : (97 , 99 ),
    'TTRACT'  : (99 , 105),
    'TBLKGRP' : (105, 106),
    'ANRC'    : (106, 111),
    'ANRCCC'  : (111, 113),
    'CBSA'    : (113, 118),
    'CBSASC'  : (118, 120),
    'METDIV'  : (120, 125),
    'CSA'     : (125, 128),
    'NECTA'   : (128, 133),
    'NECTASC' : (133, 135),
    'NECTADIV': (135, 140),
    'CNECTA'  : (140, 143),
    'CBSAPCI' : (143, 144),
    'NECTAPCI': (144, 145),
    'UA'      : (145, 150),
    'UASC'    : (150, 152),
    'UATYPE'  : (152, 153),
    'UR'      : (153, 154),
    'CD'      : (154, 156),
    'SLDU'    : (156, 159),
    'SLDL'    : (159, 162),
    'VTD'     : (162, 168),
    'VTDI'    : (168, 169),
    'RESERVE2': (169, 172),
    'ZCTA5'   : (172, 177),
    'SUBMCD'  : (177, 182),
    'SUBMCDCC': (182, 184),
    'SDELM'   : (184, 189),
    'SDSEC'   : (189, 194),
    'SDUNI'   : (194, 199),
    'AREALAND': (119, 213),
    'AREAWATR': (213, 227),
    'NAME'    : (227, 317),
    'FUNCSTAT': (317, 318),
    'GCUNI'   : (318, 319),
    'POP100'  : (319, 328),
    'HU100'   : (328, 337),
    'INTPTLAT': (337, 348),
    'INTPTLON': (348, 360),
    'LSADC'   : (360, 362),
    'PARTFLAG': (362, 363),
    'RESERVE3': (363, 369),
    'UGA'     : (369, 374),
    'STATENS' : (374, 382),
    'COUNTYNS': (382, 390),
    'COUSUBNS': (390, 398),
    'PLACENS' : (398, 406),
    'CONCITNS': (406, 414),
    'AIANHHNS': (414, 422),
    'AITSNS'  : (422, 430),
    'ANRCNS'  : (430, 438),
    'SUBMCDNS': (438, 446),
    'CD113'   : (446, 448),
    'CD114'   : (448, 450),
    'CD115'   : (450, 452),
    'SLDU2'   : (452, 455),
    'SLDU3'   : (455, 458),
    'SLDU4'   : (458, 461),
    'SLDL2'   : (461, 464),
    'SLDL3'   : (464, 467),
    'SLDL4'   : (467, 470),
    'AIANHHSC': (470, 472),
    'CSASC'   : (472, 476),
    'CNECTASC': (474, 477),
    'MEMI'    : (476, 478),
    'NMEMI'   : (477, 478),
    'PUMA'    : (478, 483),
    'RESERVED': (483, 501),
}

# https://www.census.gov/prod/cen2010/doc/sf2.pdf
FILE_1_DATA_COLS = [
    'FILEID',
    'STUSAB',
    'CHARITER',
    'CIFSN',
    'LOGRECNO',
    'TOTAL_POPULATION',
]

STATES = {
    'AL': 'Alabama',
#    'AK': 'Alaska',
#    'AZ': 'Arizona',
#     'AR': 'Arkansas',
#     'CA': 'California',
#     'CO': 'Colorado',
#     'CT': 'Connecticut',
#     'DE': 'Delaware',
#     'DC': 'District_of_Columbia',
#     'FL': 'Florida',
#     'GA': 'Georgia',
#     'HI': 'Hawaii',
#     'ID': 'Idaho',
#     'IL': 'Illinois',
#     'IN': 'Indiana',
#     'IA': 'Iowa',
#     'KS': 'Kansas',
#     'KY': 'Kentucky',
#     'LA': 'Louisiana',
#     'ME': 'Maine',
#     'MD': 'Maryland',
#     'MA': 'Massachusetts',
#     'MI': 'Michigan',
#     'MN': 'Minnesota',
#     'MS': 'Mississippi',
#     'MO': 'Missouri',
#     'MT': 'Montana',
#     'NE': 'Nebraska',
#     'NV': 'Nevada',
#     'NH': 'New_Hampshire',
#     'NJ': 'New_Jersey',
#     'NM': 'New_Mexico',
#     'NY': 'New_York',
#     'NC': 'North_Carolina',
#     'ND': 'North_Dakota',
#     'OH': 'Ohio',
#     'OK': 'Oklahoma',
#     'OR': 'Oregon',
#     'PA': 'Pennsylvania',
#     'PR': 'Puerto_Rico',
#     'RI': 'Rhode_Island',
#     'SC': 'South_Carolina',
#     'SD': 'South_Dakota',
#     'TN': 'Tennessee',
#     'TX': 'Texas',
#     'UT': 'Utah',
#     'VT': 'Vermont',
#     'VA': 'Virginia',
#     'WA': 'Washington',
#     'WV': 'West_Virginia',
#     'WI': 'Wisconsin',
#     'WY': 'Wyoming',    
}

URL_TEMPLATE_ZIP = 'https://www2.census.gov/census_2010/04-Summary_File_1/{state}/{state_abbrev}2010.sf1.zip'

# Create URLS
urls = {
    code.lower(): URL_TEMPLATE_ZIP.format(state_abbrev=code.lower(), state=name)
    for code, name
    in STATES.items()
}

In [5]:
def download_file(url):
    print('.', end='')
    with urllib.request.urlopen(url) as f:
        data = io.BytesIO(f.read())
        return data


data = {
    code: download_file(url)
    for code, url
    in urls.items()
}

.

KeyboardInterrupt: 

In [ ]:
# PROCESS
data

In [3]:
# https://www2.census.gov/census_2000/
import os
os.listdir('C:/Users/theon/OneDrive/Desktop')

['2019.10.23.ansari_poa.docx',
 '2019.10.23.ansari_poa.pdf',
 'B2FE22A.tmp',
 'desktop.ini',
 'escrow_statement.pdf',
 'holding',
 'junk.html',
 'Microsoft Edge.lnk',
 'mo2010.sf1',
 'mo2010.sf1.zip',
 'NaunheimResume2019.docx',
 'otherstuff',
 'temp',
 'TODO.txt',
 'Uplay.lnk',
 'us2010.sf1.zip']

In [14]:
with zipfile.ZipFile('C:/Users/theon/OneDrive/Desktop/mo2010.sf1.zip') as zf:
    # Filter out everything except the ZipInfo for csv we want
    target = zf.filelist[0]
    # Read that CSV into BytesIO object
    raw_data = io.BytesIO(zf.read(target))
    geo = pd.read_fwf(
        raw_data, 
        header=None,
        # Subtract 1
        colspecs=[
            (tuple_[0] - 1, tuple_[1] - 1)
            for tuple_
            in GEO_MAP_2010.values()
        ],
        dtype=str
)


geo.columns = tuple(GEO_MAP_2010.keys())
geo = geo.loc[geo.SUMLEV == '871']    


geo_orig = geo.copy()
geo = geo[['STUSAB', 'LOGRECNO', 'ZCTA5']].dropna(subset=['ZCTA5'])

geo_orig

,FILEID,STUSAB,SUMLEV,GEOCOMP,CHARITER,CIFSN,LOGRECNO,REGION,DIVISION,STATE,...,SLDL2,SLDL3,SLDL4,AIANHHSC,CSASC,CNECTASC,MEMI,NMEMI,PUMA,RESERVED
389482,SF1ST,MO,871,00,000,NaN,0389483,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389484,SF1ST,MO,871,00,000,NaN,0389485,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389486,SF1ST,MO,871,00,000,NaN,0389487,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389488,SF1ST,MO,871,00,000,NaN,0389489,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389490,SF1ST,MO,871,00,000,NaN,0389491,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389492,SF1ST,MO,871,00,000,NaN,0389493,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389494,SF1ST,MO,871,00,000,NaN,0389495,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389496,SF1ST,MO,871,00,000,NaN,0389497,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389498,SF1ST,MO,871,00,000,NaN,0389499,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389500,SF1ST,MO,871,00,000,NaN,0389501,2,4,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
with zipfile.ZipFile('C:/Users/theon/OneDrive/Desktop/mo2010.sf1.zip') as zf:
    # Filter out everything except the ZipInfo for csv we want
    target = zf.filelist[3]
    print(target.filename)
    # Read that CSV into BytesIO object
    raw_data = io.BytesIO(zf.read(target))
    data = pd.read_csv(
        raw_data, 
        header=None,
        dtype=str
    )
    pop = data[[1, 4, 18, 19, 20, 21, 22, 23, 24, 25]]
    pop.columns = [
        'STUSAB',
        'LOGRECNO',
        'white',
        'black',
        'native',
        'asian',
        'pi',
        'other',
        'multiple',
        'hispanic',
    ]
    
pop.head()

mo000032010.sf1


,STUSAB,LOGRECNO,white,black,native,asian,pi,other,multiple,hispanic
0,MO,0000001,4850748,687149,24062,97221,5763,5372,106142,212470
1,MO,0000002,0,0,0,0,0,0,0,0
2,MO,0000003,0,0,0,0,0,0,0,0
3,MO,0000004,0,0,0,0,0,0,0,0
4,MO,0000005,0,0,0,0,0,0,0,0


In [16]:
merged = geo.merge(pop)
merged = merged.set_index('ZCTA5')
merged = merged.sort_index()
merged

,STUSAB,LOGRECNO,white,black,native,asian,pi,other,multiple,hispanic
ZCTA5,,,,,,,,,,
51640,MO,0389483,11,0,0,0,0,0,0,0
52542,MO,0389485,40,0,0,0,0,0,0,0
52573,MO,0389487,25,0,0,0,0,0,0,0
52626,MO,0389489,101,0,0,0,0,0,0,0
63005,MO,0389491,15628,348,25,1061,5,15,220,451
63010,MO,0389493,33697,196,66,307,5,15,392,761
63011,MO,0389495,32319,811,52,1803,3,37,494,828
63012,MO,0389497,9615,44,26,30,2,5,92,159
63013,MO,0389499,1567,4,0,1,0,0,9,8


In [59]:
jg2 = jgeo.merge(jdata)
jg2['RACE'] = 'White'
jg2

,STUSAB,LOGRECNO,ZCTA5,TOTAL_POPULATION,RACE
0,MO,0000025,63501,0,White
1,MO,0000026,63501,0,White
2,MO,0000027,63501,0,White
3,MO,0000028,63501,0,White
4,MO,0000029,63501,31,White
5,MO,0000030,63501,2,White
6,MO,0000031,63501,1,White
7,MO,0000033,63501,0,White
8,MO,0000034,63501,0,White
9,MO,0000035,63501,19,White


PCT1: Total population
PCT0010001 

DATA FILE SEGMENT 1, CELL 1 PCT1

Iterations

001: Total Population
400: Hispanic of Latino of Any Race
450: All Not Hispanic
451: White Not Hispanic
453: Black Not Hispanic
455: Native Not Hispanic
457: Asian Not Hispanic
459: Pacific Islander Not Hispanic
463: Multiple

->

white
black
asian
native
multiple
hispanic

# Background (links good as of 2019-12-07)

This fetches data from the following zipfiles:

* https://www2.census.gov/topics/genealogy/2000surnames/names.zip
* https://www2.census.gov/topics/genealogy/2010surnames/names.zip

These zipfiles contains CSVs with race data based on the surnames. The script creates dataframes from the CSVs in these zipfiles. It then cleans them and imputes data to remove anonymization in accordance with the rules laid out below.

Information about the contents of these files may be found here:

* http://www2.census.gov/topics/genealogy/2000surnames/surnames.pdf?#
* https://www2.census.gov/topics/genealogy/2010surnames/surnames.pdf

It then the data for use in Surgeo calculations.

# Constants

In [2]:
CENSUS_URL_2000 = 'https://www2.census.gov/topics/genealogy/2000surnames/names.zip'

CENSUS_URL_2010 = 'https://www2.census.gov/topics/genealogy/2010surnames/names.zip'

CENSUS_SURNAME_COLUMNS = [
    'name',
    'rank',
    'count',
    'proportion',
    'cum_proportion',
    'white',
    'black',
    'asian',
    'native',
    'multiple',
    'hispanic',
]

TARGET_SURNAME_COLUMNS = [
    'name',
    'white',
    'black',
    'asian',
    'native',
    'multiple',
    'hispanic',
]

## Download Surname Data

In [3]:
def url_to_df(url):
    '''Takes the URL of a Census zip file and converts to DF
    
    Note: it appears the Census webservers rate limit this so it
    may take some time.
    
    '''
    # Download zipfile from census URL
    with urllib.request.urlopen(url) as response:
        # Write file into BytesIO object
        zip_data = io.BytesIO(response.read())
        # Open zip data as zipfile
        with zipfile.ZipFile(zip_data) as zf:
            # Filter out everything except the ZipInfo for csv we want
            csv_info = [file for file in zf.filelist if '.csv' in file.filename][0]
            # Read that CSV into BytesIO object
            raw_data = io.BytesIO(zf.read(csv_info))
            # Create dataframe with only suppressed '(S)' converted to NA
            df = pd.read_csv(raw_data, na_values='(S)', keep_default_na=False)
            return df

In [4]:
df_2000 = url_to_df(CENSUS_URL_2000)
df_2010 = url_to_df(CENSUS_URL_2010)

In [5]:
df_2010.tail()

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
162249,DIETZMANN,160975,100,0.03,90062.93,96.00,0.00,0.00,NaN,0.00,NaN
162250,DOKAS,160975,100,0.03,90062.96,94.00,NaN,0.00,0.00,NaN,NaN
162251,DONLEA,160975,100,0.03,90062.99,94.00,0.00,0.00,0.00,0.00,6.00
162252,DORIOTT,160975,100,0.03,90063.03,89.00,0.00,NaN,0.00,5.00,NaN
162253,ALL OTHER NAMES,0,29312001,9936.97,9936.97,66.65,8.53,7.97,0.86,2.32,13.67


# Clean Data

In [6]:
def clean_df(df):
    '''Change column names, set index, and convert percentages'''
    # Change names
    df.columns = CENSUS_SURNAME_COLUMNS
    # Filter columns
    df = df[TARGET_SURNAME_COLUMNS]
    # Set index to name
    df = df.set_index('name')
    # Sort index
    df = df.sort_index()
    # Convert percentages to 0 to 1 numbers
    df = df / 100
    return df

In [7]:
df_2000 = clean_df(df_2000)
df_2010 = clean_df(df_2010)

In [8]:
df_2010.tail()

,white,black,asian,native,multiple,hispanic
name,,,,,,
ZYSK,0.9873,0.0,NaN,NaN,0.0000,NaN
ZYSKOWSKI,0.9655,NaN,NaN,0.0,0.0164,0.0127
ZYSMAN,0.9457,NaN,0.0,0.0,NaN,NaN
ZYWICKI,0.9552,NaN,NaN,0.0,0.0125,0.0233
ZYWIEC,0.9829,NaN,0.0,0.0,0.0000,NaN


# Unsuppress / impute anonymized data

In [9]:
def unsuppress_row(row):
    '''Apply function to desuppress data on row-basis
    
    If a percentage falls beneath a certain threshold, the
    US Census quasi-anonymizes it by supressing. To impute
    new values, we get the outstanding allocated percentage
    and divy it up among the suppressed fields.
    
    '''
    # Check if row has NA values
    if row.isna().sum() > 0:
        # Get count of NA values
        na_count = row.isna().sum()
        # Get total of percentages
        row_sum = row.sum()
        # Get unallocated percentage and divide by count of NA
        na_value = (1 - row_sum) / na_count
        # Fill NA values with that row value.
        reconstituted_row = row.fillna(na_value)
        # Round if necessary
        return reconstituted_row.round(4)
    else:
        # If there's no NA, there's no need to impute
        return row

In [10]:
# Get rows with NaNs
target_2000 = df_2000.isna().any(axis=1)
# Run this inefficient operation on rows with NaNs only
df_2000.loc[target_2000] = (
    df_2000.loc[target_2000].apply(unsuppress_row, axis=1)
)

# Get rows with NaNs
target_2010 = df_2010.isna().any(axis=1)
# Run this inefficient operation on rows with NaNs only
df_2010.loc[target_2010] = (
    df_2010.loc[target_2010].apply(unsuppress_row, axis=1)
)

In [11]:
df_2010.tail()

,white,black,asian,native,multiple,hispanic
name,,,,,,
ZYSK,0.9873,0.0000,0.0042,0.0042,0.0000,0.0042
ZYSKOWSKI,0.9655,0.0027,0.0027,0.0000,0.0164,0.0127
ZYSMAN,0.9457,0.0181,0.0000,0.0000,0.0181,0.0181
ZYWICKI,0.9552,0.0045,0.0045,0.0000,0.0125,0.0233
ZYWIEC,0.9829,0.0085,0.0000,0.0000,0.0000,0.0085


# Write data to module as CSV

In [12]:
current_directory = pathlib.Path().cwd()
project_directory = current_directory.parents[0]
data_directory    = project_directory / 'surgeo' / 'data'
path_2000         = data_directory / 'surnames_2000.csv'
path_2010         = data_directory / 'surnames_2010.csv'
df_2000.to_csv(path_2000)
df_2010.to_csv(path_2010)